In [1]:
import torch
from torch.nn import functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Prepration

In [3]:
from torchtext.data import get_tokenizer

In [4]:
#pytorch tokenizer
tokenizer = get_tokenizer("basic_english")

In [5]:
#load glove embeddings 
from torchtext.vocab import GloVe

global_vectors = GloVe(name='6B', dim=100)

In [6]:
vocab = global_vectors.stoi

# Determine the number of unique tokens
num_tokens = len(vocab)
num_tokens

400000

In [7]:
len(global_vectors.vectors)

400000

In [8]:
glove_embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"), lower_case_backup=True)
print(tokenizer("Hello, How are you?"))
glove_embeddings.shape

['hello', ',', 'how', 'are', 'you', '?']


torch.Size([6, 100])

In [9]:
import pandas as pd

train_df = pd.read_csv('data/train.tsv', delimiter='\t', header=None, names=['unique_id', 'label', 'extra_label', 'text'])
train_df

,unique_id,label,extra_label,text
0,25462,1,a,There are not any jew signatures on our Declar...
1,22529,1,a,"Hey ,muslim dickhead !!!!!!!! Why don;t you go..."
2,29861,0,a,"Generally true, but there are indeed missionar..."
3,6358,1,a,Why do so many filthy wetback half-breed spic ...
4,7907,0,a,Why are girl so ugly to the girls that are jus...
...,...,...,...,...
34234,6198,0,a,Why are Hindus always an easy target. On my re...
34235,2100,0,a,"You people should be proud to be gay,lesbian,b..."
34236,14780,1,a,i hate jews so much i wish the holocaust actua...
34237,8837,1,a,ayo i even kill handicapped and crippled bitch...


In [10]:
sentences = train_df.text.values
labels = train_df.label.values

In [11]:
tokenized_texts = [tokenizer(sent) for sent in sentences]

In [12]:
unique_tokens = set(token for tokens in tokenized_texts for token in tokens)
vocab_size = len(unique_tokens)
vocab_size

20886

In [13]:
max_words = 25
pad_tokenized_text = [tokens + [""] * (max_words - len(tokens)) if len(tokens) < max_words else tokens[:max_words] for tokens in tokenized_texts]

In [14]:
pad_tokenized_text

[['there',
  'are',
  'not',
  'any',
  'jew',
  'signatures',
  'on',
  'our',
  'declaration',
  'of',
  'independence',
  'those',
  'subversive',
  'babylonian',
  'blood',
  'sucking',
  'serpent',
  'seed',
  'kenite',
  'vampires',
  'are',
  'not',
  'out',
  'founding',
  'fathers'],
 ['hey',
  ',',
  'muslim',
  'dickhead',
  '!',
  '!',
  '!',
  '!',
  '!',
  '!',
  '!',
  '!',
  'why',
  'don',
  't',
  'you',
  'go',
  'and',
  'shove',
  'allah',
  'or',
  'whatever',
  'the',
  'fuck',
  'you'],
 ['generally',
  'true',
  ',',
  'but',
  'there',
  'are',
  'indeed',
  'missionaries',
  'in',
  'many',
  'of',
  'these',
  'areas',
  ',',
  'and',
  'they',
  'do',
  'indeed',
  'teach',
  '.',
  'for',
  'example',
  ',',
  'there',
  'are'],
 ['why',
  'do',
  'so',
  'many',
  'filthy',
  'wetback',
  'half-breed',
  'spic',
  'savages',
  'live',
  'in',
  '#losangeles',
  '?',
  'none',
  'of',
  'them',
  'have',
  'any',
  'right',
  'at',
  'all',
  'to',
  'be',

In [15]:
from torchtext.vocab import vocab
unk_token = "<unk>"
unk_index = 0

glove_vocab = vocab(global_vectors.stoi)
glove_vocab.insert_token("<unk>",unk_index)
#this is necessary otherwise it will throw runtime error if OOV token is queried 
glove_vocab.set_default_index(unk_index)

In [16]:
pad_tokenized_text = [glove_vocab(tokens) for tokens in pad_tokenized_text]

In [17]:
pad_tokenized_text

[[63,
  32,
  36,
  130,
  12471,
  9980,
  13,
  162,
  3616,
  3,
  1372,
  155,
  21443,
  33099,
  1454,
  27383,
  24980,
  3404,
  0,
  23017,
  32,
  36,
  66,
  3968,
  8669],
 [7942,
  1,
  1172,
  0,
  805,
  805,
  805,
  805,
  805,
  805,
  805,
  805,
  738,
  3316,
  2159,
  81,
  242,
  5,
  29816,
  14272,
  46,
  2684,
  0,
  35690,
  81],
 [1573,
  1446,
  1,
  34,
  63,
  32,
  2747,
  10923,
  6,
  109,
  3,
  158,
  619,
  1,
  5,
  39,
  88,
  2747,
  5293,
  2,
  10,
  880,
  1,
  63,
  32],
 [738,
  88,
  100,
  109,
  25939,
  234867,
  152412,
  109096,
  46350,
  682,
  6,
  0,
  188,
  2125,
  3,
  101,
  33,
  130,
  248,
  22,
  64,
  4,
  30,
  187,
  2],
 [738,
  32,
  1749,
  100,
  8094,
  4,
  0,
  1946,
  12,
  32,
  120,
  595,
  4,
  7019,
  101,
  41,
  57,
  31719,
  332,
  1906,
  40488,
  0,
  0,
  0,
  0],
 [35690,
  81,
  35690,
  81,
  81,
  8979,
  12478,
  30619,
  242,
  28481,
  7,
  4159,
  5,
  2357,
  10574,
  7,
  8979,
  42443,
  3

In [18]:
#pad_tokenized_text = [global_vectors.get_vecs_by_tokens(tokens).tolist() for tokens in pad_tokenized_text]

In [19]:
pad_tokenized_text = torch.tensor(pad_tokenized_text)
labels = torch.tensor(labels, dtype=torch.long)

In [20]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

batch = 32
embed_len = 100


train_data = TensorDataset(pad_tokenized_text, labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch)

In [21]:
len(train_dataloader)

1070

# Define Model

In [22]:
class SelfAttention(torch.nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        
        #query matrix
        self.Wq = torch.nn.Parameter(torch.randn(input_dim, input_dim))
        #key matrix
        self.Wk = torch.nn.Parameter(torch.randn(input_dim, input_dim))
        #value matrix
        self.Wv = torch.nn.Parameter(torch.randn(input_dim, input_dim))
        
        #for normalization
        self.softmax = torch.nn.Softmax(dim=2)
        
    def forward(self, x):
        #x = x.transpose(0, 1) # Assuming x is of shape (sequence_length, batch_size, input_dim)
        #print(x.shape)
        #print(self.Wq.shape)
        queries = torch.matmul(x, self.Wq)
        keys = torch.matmul(x, self.Wk)
        values = torch.matmul(x, self.Wv)
        
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        
        weighted = weighted.transpose(0, 1)
        
        return weighted 

In [23]:
class customeModel(torch.nn.Module):
    def __init__(self, glove_vectors, input_dim, hidden_size, embedding_dimensions, num_classes):
        super(customeModel, self).__init__()
        
        #variables
        self.glove_vectors = glove_vectors
        self.input_dim = input_dim
        self.embedding_dimensions = embedding_dimensions
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        #layer
        self.embedding_layer = torch.nn.Embedding.from_pretrained(self.glove_vectors.vectors, freeze=True, sparse=True)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm_1 = torch.nn.LSTM(self.embedding_dimensions, self.hidden_size, batch_first = True)
        
        self.attention = SelfAttention(self.input_dim)
        
        self.lstm_2 = torch.nn.LSTM(self.hidden_size, self.hidden_size, batch_first = True)
        
        self.output_layer = torch.nn.Linear(self.hidden_size, self.num_classes)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, batch_input):
        embeds = self.embedding_layer(batch_input)
        lstm_out_1, _ = self.lstm_1(embeds)
        attn_out = self.attention(lstm_out_1)
        lstm_out_2, _ = self.lstm_2(attn_out)
        output = self.output_layer(lstm_out_2) #lstm_out_2[:, -1, :] using last hidden_state for classification
        output_sigmoid = self.sigmoid(output)
        
        #return F.log_softmax(output, dim=1)
        return output_sigmoid
        

# Train Model

In [24]:
from torch.optim import Adam

epochs = 25
learning_rate = 1e-3

loss_fn = torch.nn.CrossEntropyLoss()
custom_model = customeModel(glove_vectors = global_vectors, #glove vectors
                            input_dim = 25, #self-attention
                            hidden_size = 25, #lstm cell
                            embedding_dimensions = 100, #glove embedding 
                            num_classes = 1) #output classes are 2 but neurons req out_class - 1

optimizer = Adam(custom_model.parameters(), lr=learning_rate)

In [25]:
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch

In [26]:
print(len(b_input_ids))
print(b_input_ids.shape)

31
torch.Size([31, 25])


In [27]:
print(len(b_labels))
print(b_labels.shape)

31
torch.Size([31])


In [28]:
b_input_ids[0]

tensor([  169,  6642,     3,   155, 48271,  2479,   805,     0, 10764,  2001,
           12,   227,    57,  2159, 40181,     1,  4693,   266,     1,   960,
          271,     1,  6096,  1517,     1])

In [29]:
custom_model.zero_grad()

In [30]:
outputs = custom_model(b_input_ids)

In [31]:
outputs

tensor([[[0.4852],
         [0.4780],
         [0.4787],
         [0.4818],
         [0.4811],
         [0.4777],
         [0.4810],
         [0.4814],
         [0.4781],
         [0.4833],
         [0.4852],
         [0.4823],
         [0.4776],
         [0.4845],
         [0.4778],
         [0.4837],
         [0.4775],
         [0.4807],
         [0.4797],
         [0.4844],
         [0.4734],
         [0.4784],
         [0.4769],
         [0.4741],
         [0.4799],
         [0.4772],
         [0.4775],
         [0.4830],
         [0.4805],
         [0.4797],
         [0.4856]],

        [[0.4854],
         [0.4783],
         [0.4785],
         [0.4810],
         [0.4808],
         [0.4778],
         [0.4802],
         [0.4804],
         [0.4782],
         [0.4820],
         [0.4854],
         [0.4816],
         [0.4771],
         [0.4859],
         [0.4779],
         [0.4836],
         [0.4783],
         [0.4787],
         [0.4797],
         [0.4858],
         [0.4730],
         [

In [32]:
outputs.shape

torch.Size([25, 31, 1])

In [33]:
class_probs = torch.mean(outputs, dim=0)
print(class_probs.shape)
print(class_probs)

torch.Size([31, 1])
tensor([[0.4852],
        [0.4776],
        [0.4777],
        [0.4808],
        [0.4810],
        [0.4774],
        [0.4805],
        [0.4818],
        [0.4780],
        [0.4817],
        [0.4848],
        [0.4828],
        [0.4773],
        [0.4842],
        [0.4776],
        [0.4832],
        [0.4774],
        [0.4785],
        [0.4791],
        [0.4842],
        [0.4726],
        [0.4781],
        [0.4775],
        [0.4754],
        [0.4784],
        [0.4767],
        [0.4766],
        [0.4816],
        [0.4797],
        [0.4774],
        [0.4842]], grad_fn=<MeanBackward1>)


In [34]:
#predicted_classes = torch.argmax(class_probs, dim=1)
#print(predicted_classes)
#print(len(predicted_classes))
#predicted_classes

In [35]:
#max_values, predicted_classes = torch.max(class_probs, dim=1)
#print(max_values)
#predicted_classes

In [36]:
len(b_labels)

31

In [37]:
#predicted_classes = predicted_classes.float()
#b_labels = b_labels.float()

In [38]:
b_labels_2d = b_labels.view(-1, 1)
b_labels_2d = b_labels_2d.float()

loss_function = torch.nn.BCELoss()
loss = loss_function(class_probs, b_labels_2d)

In [39]:
loss.backward()

In [40]:
optimizer.step()

# Automate Training

In [41]:
from tqdm import tqdm


for epochs in tqdm(range(10)):
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_labels = batch
        
        #clear gradients 
        custom_model.zero_grad()
        
        #get output and transform it according to labels 
        outputs = custom_model(b_input_ids)
        class_probs = torch.mean(outputs, dim=0)
        
        #reshape labels and convert to float 
        b_labels_2d = b_labels.view(-1, 1)
        b_labels_2d = b_labels_2d.float()
        
        loss = loss_function(class_probs, b_labels_2d)
        print("Loss " + str(loss))
        #compute gradient 
        loss.backward()
        
        #update parameters 
        optimizer.step()

  0%|                                                                                                                                         | 0/10 [00:00<?, ?it/s]

Loss tensor(0.6904, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6899, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6810, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6939, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.7045, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6982, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6893, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6885, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6768, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6782, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6679, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6692, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.7150, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.7033, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6899, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.7084, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.6938, grad_fn=<BinaryCrossEntropyBackward0

Loss tensor(0.0844, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1430, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0806, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1656, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1979, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1262, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0938, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2961, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1938, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2156, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1058, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1069, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1341, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1682, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0937, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2915, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1547, grad_fn=<BinaryCrossEntropyBackward0

Loss tensor(0.0326, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1884, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.3245, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1297, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1214, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1143, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1759, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1965, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.3049, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0696, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0513, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0885, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0568, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2524, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0628, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0734, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1812, grad_fn=<BinaryCrossEntropyBackward0

Loss tensor(0.1542, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1671, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1120, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0795, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1299, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1757, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2970, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0328, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1687, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1075, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2012, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1302, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0290, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0258, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1885, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1525, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0519, grad_fn=<BinaryCrossEntropyBackward0

...

Loss tensor(0.0266, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0042, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0089, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0006, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.2482, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0006, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0018, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1343, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0152, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0012, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0271, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0043, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0026, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0011, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0077, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0708, grad_fn=<BinaryCrossEntropyBackward0

Loss tensor(0.0328, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0178, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0051, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0054, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0067, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0007, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0135, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0066, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0049, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0194, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0039, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0023, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0012, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0242, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0121, grad_fn=<BinaryCrossEntropyBackward0

Loss tensor(0.2464, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0162, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0664, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0033, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0006, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0012, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0120, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0007, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0011, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0019, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0007, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0009, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0009, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0014, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0007, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0007, grad_fn=<BinaryCrossEntropyBackward0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:15<00:00,  7.54s/it]

Loss tensor(0.0147, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0147, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.1515, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0042, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0310, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0008, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward0>)
Loss tensor(0.0067, grad_fn=<BinaryCrossEntropyBackward0>)


In [42]:
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch
    
with torch.no_grad():
    outputs = custom_model(b_input_ids)
    class_probs = torch.mean(outputs, dim=0)
    b_labels_2d = b_labels.view(-1, 1)
    b_labels_2d = b_labels_2d.float()
        
    loss = loss_function(class_probs, b_labels_2d)

In [43]:
loss

tensor(0.0027)

# Test - Hate

In [44]:
test_sentence = 'Fuck this bitch'

tokenized_test_sentence = tokenizer(test_sentence)
print(tokenized_test_sentence)

['fuck', 'this', 'bitch']


In [50]:
max_words = 25

def pad_sentence(sentence):
    if len(sentence) < max_words:
        
        sentence = sentence + [""]*(max_words - len(sentence))
        
    else:
        sentence = sentence[:max_words]
        
    return sentence

In [52]:
pad_tokenized_test_sentence = pad_sentence(tokenized_test_sentence)

In [61]:
pad_tokenized_test_sentence = glove_vocab(pad_tokenized_test_sentence)

In [62]:
pad_tokenized_test_sentence = torch.tensor(pad_tokenized_test_sentence)

In [65]:
pad_tokenized_test_sentence.shape

torch.Size([25])

In [68]:
pad_tokenized_test_sentence = pad_tokenized_test_sentence.view(1, -1)
pad_tokenized_test_sentence.shape

torch.Size([1, 25])

In [69]:
custom_model.eval()
output = custom_model(pad_tokenized_test_sentence)

In [70]:
output

tensor([[[0.9969]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]],

        [[0.9974]]], grad_fn=<SigmoidBackward0>)

In [71]:
torch.mean(output, dim=0)

tensor([[0.9974]], grad_fn=<MeanBackward1>)

# Test - Not Hate

In [72]:
test_sentence = 'She is a good girl'
tokenized_test_sentence = tokenizer(test_sentence)

In [73]:
#pad
pad_tokenized_test_sentence = pad_sentence(tokenized_test_sentence)

#word - index
pad_tokenized_test_sentence = glove_vocab(pad_tokenized_test_sentence)

#tensor
pad_tokenized_test_sentence = torch.tensor(pad_tokenized_test_sentence)

In [74]:
#shape
pad_tokenized_test_sentence = pad_tokenized_test_sentence.view(1, -1)
pad_tokenized_test_sentence.shape

torch.Size([1, 25])

In [75]:
custom_model.eval()
output = custom_model(pad_tokenized_test_sentence)

torch.mean(output, dim=0)

tensor([[0.0013]], grad_fn=<MeanBackward1>)